## Imports

In [ ]:
#Commonted out the below lines as the packages are already installed in the environment

#!pip install timm
#!pip install nibabel
#!pip install nilearn

import tensorflow as tf
#for neuroimaging data
import nibabel as nib
import nilearn as nilearn
import timm as timm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [ ]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

In [ ]:
# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {torch.cuda.get_device_name()} for inference')

## Importing the Dataset

In [ ]:
import os
import nibabel as nib

root_dir = 'brown'

# List all subdirectories in the root directory
subdirectories = [x[0] for x in os.walk(root_dir)]

dataset = []

# Iterate over the subdirectories
for subdir in subdirectories:
    # Check if the subdirectory contains 'anat' in its path
    if 'anat' in subdir:
        # List all files in the subdirectory
        files = os.listdir(subdir)
        # Iterate over the files
        for file in files:
            # Check if the file is a .nii.gz file
            if file.endswith('.nii.gz'):
                file_path = os.path.join(subdir, file)
                # Load the Nifti image
                data = nib.load(file_path)
                # Append the image data to the dataset
                dataset.append(data)
         
# Print the number of images in the dataset
print('Number of Images in Dataset: ' + str(len(dataset)))

## Data Profiling

In [ ]:
# Access data array and metadata
data = dataset[22].get_fdata()
affine = dataset[22].affine 
header = dataset[22].header

# Display basic information
print(f'Image dimensions: {data.shape}')
print(f'Voxel dimensions: {header.get_zooms()}')

In [ ]:
# Display the image

plt.imshow(data[:, :, 90], cmap='gray')
plt.axis('on')



## Preprocessing

In [ ]:
# Resample the dataset

from nilearn.image import resample_img

# Define the target voxel dimensions
target_voxel_dimensions = (1.0, 1.0, 1.0)

# Resample the dataset to the target voxel dimensions
resampled_dataset = []
for data in dataset:
    resampled_data = resample_img(data, target_affine=np.diag(target_voxel_dimensions))
    resampled_dataset.append(resampled_data)

# Display the resampled image
resampled_data = resampled_dataset[22].get_fdata()
plt.imshow(resampled_data[:, :, 90], cmap='gray')
plt.axis('on')

# Load the labels
labels = pd.read_csv('brown/participants.tsv')
labels.head()


## Loading the Pre-Trained ResNet Model

In [ ]:
import torch
from torchvision.models.video import r3d_18

# Load the pre-trained model
model = r3d_18(pretrained=True)

# Modify the last layer
num_classes = 2  # replace with the number of classes in your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

# Print the model
model

In [ ]:
# from sklearn.model_selection import train_test_split

# # Split the dataset into training and validation sets
# train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

# # Print the number of images in the training and validation sets
# print('Number of Images in Training Set: ' + str(len(train_dataset)))
# print('Number of Images in Validation Set: ' + str(len(val_dataset)))